# Preamp

The preamplification stage converts a photo current $I_\text{photo}$ to a voltage signal $V_\text{out}$ using a transimpedance amplifier circuit.
Conversion is determined by,
$$
V_\text{out}=-R_\text{f}I_\text{photo},
$$
whereing $R_\text{f}$ is the value of the feedback resistor $R_\text{f}$.

In this notebook we want to find the optimal parameter of $R_\text{f}$ respective $V_\text{out}$ given,
* an operating temperature $T$,
* an operating bandwidth $\Delta f$,
* a maximum photo current $I^\max_\text{photo}$,
* and a dual supply voltage $V_\text{supply}$.

As an upper bound we have $\vert V_\text{out}\vert <V_\text{supply}$ because we want don't want to operate the op-amp rail-to-rail.
If we choose $V_\text{out}$ to be greater than the optimum, then the amplified noise will be dominant.
However, if we choose $V_\text{out}$ to be less than the optimum, then unamplified (background) noise will be dominant.

In [1]:
using Unitful

struct Requirements
    bandwidth::Unitful.Frequency
    photocurrent::Unitful.Current
    supplyvoltage::Unitful.Voltage
    temperature::Unitful.Temperature
end

requirements = Requirements(1u"kHz", 100u"µA", 15u"V", 300u"K")

Requirements(1 kHz, 100 μA, 15 V, 300 K)

## Noise

We now give an overview of the different noise sources inside the transimpedance amplifier and their respective contribution to the output voltage signal.
Note that although some noises are small at their occurance but large after amplification.

### Thermal (Johnson-Nyquist) noise

Thermal noise arises from thermal fluctuations in equilibrium.
It increases with temperature and resistance as a higher resistance requires higher heat dissipation.
$$\Delta V_\text{thermal}=\sqrt{4k_BTR\Delta f}$$
The former equation holds true for any resistors $R$ not subject to amplification.
Resistors in close proximity to the input terminals of the op-amp contribute a current,
$$\Delta I_\text{thermal}=\Delta V_\text{thermal}/R=\sqrt{4k_BT\Delta f/R},$$
which is contributes to the output voltage $V_\text{out}$ in the same way as the photo current $I_\text{photo}$.

In [7]:
function thermal_current_noise(resistance::Unitful.ElectricalResistance) ::Unitful.Current
    return (4 * Unitful.k * requirements.temperature * requirements.bandwidth / resistance)^(1/2)
end

thermal_current_noise (generic function with 1 method)

### Shot (Poisson) noise

Shot noises arises from the discrete nature of the electron excitation at the photodiode junction.
$$\Delta I_\text{shot}=\sqrt{2e_0(I_\text{dark}+I_\text{photo})\Delta f}$$
As a current it contributes to the output voltage $V_\text{out}$ in the same way as the photo current $I_\text{photo}$.

In [11]:
function shot_current_noise(darkcurrent::Unitful.Current) ::Unitful.Current
    return (2 * Unitful.q * (darkcurrent + requirements.photocurrent) * requirements.bandwidth)^(1/2)
end

shot_current_noise (generic function with 1 method)

### Non-ideal op-amp

For a non-ideal op-amp we need to consider,
* finite gain,
* finite bandwidth,
* finite input impedance,
* and input offset voltage.

We will neglect further imperfections.